In [1]:
#import
import os
from functools import partial
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist
import warnings
from pandas.errors import SettingWithCopyWarning
from pyro.infer import MCMC, NUTS

# for CI testing
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.4')
pyro.set_rng_seed(1)


# Set matplotlib settings
%matplotlib inline
plt.style.use('default')

In [2]:
# Extract morphing level and shock (binary morphing level)

# Equal for all subjects
df = pd.read_csv('../data/newLookAtMe/newLookAtMe20.csv')
data = df[['morphing level', 'shock']]
data['shock'] = data['shock'].astype(int)
data['morphing level'] = [int(d==6) for d in data['morphing level']]

Create tensor where:
* `[0 0] = 0`
* `[1 0] = 1`
* `[1 1] = 2`

In [3]:
data_model = data.to_numpy()
data_final = []
for x in data_model:
    if (x == [0, 0]).all():
        data_final.append(0)
    elif (x == [1, 0]).all():
        data_final.append(1)
    else:
        data_final.append(2)

data_final = torch.tensor(data_final)
data_model_tensor = torch.tensor(data_model)
data_model_tensor

tensor([[0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 1],
        [0, 0],
        [0, 0],
        [1, 1],
        [0, 0],
        [1, 1],
        [1, 1],
        [0, 0],
        [1, 0],
        [1, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 1],
        [1, 0],
        [0, 0],
        [1, 1],
        [1, 1],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 1],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 1],
        [0, 0],
        [1, 1],
        [0, 0],
        [1, 1],
        [0, 0],
        [0, 0],
        [1, 1],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 1],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        

$p(data\, final | \theta) = \prod_i^3\theta_k^{N_k}$

where $\theta$ is a vector 3-dimensional modeled with a Dirichlet Distribution

In [4]:
data_model

array([[0, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0,

In [9]:
# model definition

# uniform prior
prior_counts = torch.ones((2,2))

def simple_model(data):
    theta = pyro.sample("theta", dist.Dirichlet(prior_counts))
    total_counts = int(data.sum())
    pyro.sample("likelihood", dist.Multinomial(total_counts, theta), obs=data)

counter = torch.tensor(np.unique(data_model, axis=0, return_counts=True)[1])
counter_prova = torch.tensor([[28, 0], [5,15]])

In [10]:
nuts_kernel = NUTS(simple_model)
num_samples, warmup_steps = (1000, 200) if not smoke_test else (10, 10)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200)
mcmc.run(counter_prova)
hmc_samples = {k: v.detach().cpu().numpy()
               for k, v in mcmc.get_samples().items()}

Sample: 100%|██████████| 1200/1200 [00:11, 105.02it/s, step size=5.51e-01, acc. prob=0.932]


In [34]:
not_agg_data = data_model_tensor[data_model_tensor[:,0] == 0]
agg_data = data_model_tensor[data_model_tensor[:,0] == 1]


In [39]:
data_model_tensor.shape

torch.Size([160, 2])

In [45]:
np.unique(agg_data, return_counts=True, axis=0)[1]

array([28, 28])

In [25]:
np.unique(agg_data, return_counts=True, axis=0)[1].sum()

56

In [24]:
np.append(np.unique(not_agg_data, return_counts=True, axis=0)[1], 0).sum()

104

In [84]:
prior_aggressive = torch.ones(2)
prior_not_aggressive = torch.ones(2)

def model(data):
    not_agg_data = data[data[:,0] == 0]
    agg_data = data[data[:,0] == 1]

    count_values_1 = torch.tensor(np.append(np.unique(not_agg_data, return_counts=True, axis=0)[1], 0))
    count_values_2 = torch.tensor(np.unique(agg_data, return_counts=True, axis=0)[1])

    theta1 = pyro.sample("theta1", dist.Dirichlet(prior_aggressive))
    theta2 = pyro.sample("theta2", dist.Dirichlet(prior_not_aggressive))

    counts1 = int(count_values_1.sum())
    counts2 = int(count_values_2.sum())

    pyro.sample("likelihood1", dist.Multinomial(counts1, theta1), obs = count_values_1)
    pyro.sample("likelihood2", dist.Multinomial(counts2, theta2), obs = count_values_2)



In [85]:
nuts_kernel = NUTS(model)
num_samples, warmup_steps = (1000, 200) if not smoke_test else (10, 10)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200)
mcmc.run(data_model_tensor[:48])
hmc_samples = {k: v.detach().cpu().numpy()
               for k, v in mcmc.get_samples().items()}

Sample: 100%|██████████| 1200/1200 [00:14, 80.40it/s, step size=8.40e-01, acc. prob=0.901] 


In [86]:
dio1 = hmc_samples['theta1'].mean(axis=0)
dio2 = hmc_samples['theta2'].mean(axis=0)
print(dio1)
print(dio2)

[0.96038014 0.0396203 ]
[0.49686593 0.503134  ]


La cella sopra indica le probabilità di vedere una coppia `[0 0]`, una coppia `[1 0]` o una coppia `[1 1]`, ma non $P(shock | morph\, level)$.

In [183]:
torch.tensor([dio2[1]])

tensor([0.5031])

In [190]:
x_test = torch.tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1])
unique_values = torch.unique(x_test, return_counts = True)[1]
shock = unique_values[1]

# predictive marcio
results = torch.tensor([])

results

tensor([0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.])